# Масштабирование обучение нейронной сети с помощью Horovod

[Horovod](https://github.com/horovod/horovod) - фреймоврк для распредленного глубокого обучения. Он работает с библиотеками TensorFlow, Keras, PyTorch, и Apache MXNet.Далее мы покажем как использование Horovod, разделяющего датасет на несколько GPU, ускорит обучение.

## Сценарий 1

Прежде чем начинать модификацию с целью распареллелить последовательное обучение, сначала убедимся в том, что можем обучить сеть на одном GPU. Сделаем лишь пару эпох с относительно большим размером батча.

In [15]:
!horovodrun -np 1 python artists_resnet.py --epochs 10 --batch-size 16

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 562s 709ms/step - loss: 1.4954 - accuracy: 0.4665 - val_loss: 3.2276 - val_accuracy: 0.0727
[0]<stdout>:Image/sec: 1231
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 563s 469ms/step - loss: 0.9943 - accuracy: 0.6743 - val_loss: 3.1865 - val_accuracy: 0.0765
[0]<stdout>:Image/sec: 1316
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [============

### Инициализация Horovod и выбор GPU для запуска

С Horovod, который может запускать несколько процессов на нескольких графических процессорах, вы обычно используете один графический процессор для каждого процесса обучения нейронной сети. Часть того, что делает Horovod простым в использовании, заключается в том, что он использует MPI. Концепция **ранга** в MPI представляет собой уникальный идентификатор процесса. Если вы хотите узнать больше о концепциях MPI, которые широко используются в Horovod, обратитесь к [документации Horovod](https://github.com/horovod/horovod/blob/master/docs/concepts.rst).

Схематически давайте посмотрим, как MPI может запускать несколько процессов GPU на нескольких узлах. Обратите внимание, как каждый процесс или ранг привязан к конкретному графическому процессору:

<img src="https://user-images.githubusercontent.com/16640218/53518255-7d5fc300-3a85-11e9-8bf3-5d0e8913c14f.png" width="400"></img>

`horovodrun` — это скрипт, который запускает N копий обучающего скрипта, где N — аргумент `-np`. (Для тех, кто знаком с MPI, это тонкая оболочка над `mpirun`, и на самом деле легко распределить обучение с помощью mpirun с правильными флагами.) Мы будем использовать его для координации процесса обучения. Поскольку процессы запускаются в среде MPI, они могут взаимодействовать друг с другом через стандартизированный API, который Horovod обрабатывает за нас, хотя мы еще не указали обучающему сценарию фактическую координацию;

## Сценарий 2

– Обучение с использованием 4 GPU на одном узле

In [ ]:
!horovodrun -np 4 python artists_resnet.py --epochs 10 --batch-size 64

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 85s 508ms/step - loss: 1.4900 - accuracy: 0.4526 - val_loss: 3.5123 - val_accuracy: 0.0727
[0]<stdout>:Image/sec: 784
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 63s 363ms/step - loss: 0.9946 - accuracy: 0.6977 - val_loss: 3.1695 - val_accuracy: 0.0765
[0]<stdout>:Image/sec: 2678
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [===============

Скопировать код в Azure Storage:

!cp artists_resnet.py /rapids/artists_resnet.py 

## Сценарий 3

– Обучение с использованием 4 GPU на одном узле на всех узлах (на одном из узлов 2 GPU, так как у нас всего 3 узла в кластере)

In [3]:
!horovodrun  --np 4 -H '10.244.2.4':2,'10.244.1.5':1,'10.244.0.10':1 python  python AuthorsClassificationWithHorovod.py

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 146s 753ms/step - loss: 1.4900 - accuracy: 0.7226 - val_loss: 3.2295 - val_accuracy: 0.0727
[0]<stdout>:Image/sec: 631
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 146s 476ms/step - loss: 0.4346 - accuracy: 0.7327 - val_loss: 3.1880 - val_accuracy: 0.0765
[0]<stdout>:Image/sec: 1916
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [=============

## Сценарий 4

– Обучение с использованием 12 GPU (по 4 GPU на 3 узлах)

In [1]:
!horovodrun -np 12 -H '10.244.2.4':4,'10.244.1.5':4,'10.244.0.10':4 python /rapids/artists_resnet.py --epochs 10 --batch-size 64

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 70s 753ms/step - loss: 1.4900 - accuracy: 0.7265 - val_loss: 3.2254 - val_accuracy: 0.0723
[0]<stdout>:Image/sec: 15231
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 70s 476ms/step - loss: 0.4346 - accuracy: 0.7354 - val_loss: 3.1865 - val_accuracy: 0.0744
[0]<stdout>:Image/sec: 15316
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [============